## GOBAI-O2-v1.0 (vertical mean: 0-200m; 200-600m)

In [1]:
import os
import xarray as xr
import numpy as np
import netCDF4
import seawater as sw
USER = os.environ['USER']

### EN4 T Obs

In [2]:
## monthly gridded T data; 1954-2017
path = f'/glade/u/home/{USER}/O2prediction/Prediction/DATA'
filename = f'{path}/EN4_Temp_15-65N_190-40W_monthly_1954-2017.nc'
nc = netCDF4.Dataset(filename, 'r')
z_r = nc.variables['z_t'][:]
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
temp = nc.variables['T'][:,:,:,:]
nc.close()

In [3]:
z_r

masked_array(data=[  5.0215898,  15.07854  ,  25.16046  ,  35.27829  ,
                    45.44776  ,  55.691494 ,  66.041985 ,  76.54591  ,
                    87.27029  ,  98.31118  , 109.806175 , 121.9519   ,
                   135.02855  , 149.43373  , 165.72845  , 184.69746  ,
                   207.42545  , 235.38617  , 270.53412  , 315.37408  ,
                   372.96545  , 446.80093  , 540.5022   , 657.32294  ],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

### FOSI hindcast

In [4]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
z_t = dset['z_t']/1e2 # m
#dz = dset['dz']/1e2 # m
z_t
#darea = dset['TAREA']/1e10 #km*km
#dvol = dz*darea

<xarray.DataArray 'z_t' (z_t: 60)>
array([5.0000000e+00, 1.5000000e+01, 2.5000000e+01, 3.5000000e+01,
       4.5000000e+01, 5.5000000e+01, 6.5000000e+01, 7.5000000e+01,
       8.5000000e+01, 9.5000000e+01, 1.0500000e+02, 1.1500000e+02,
       1.2500000e+02, 1.3500000e+02, 1.4500000e+02, 1.5500000e+02,
       1.6509840e+02, 1.7547905e+02, 1.8629128e+02, 1.9766028e+02,
       2.0971138e+02, 2.2257828e+02, 2.3640883e+02, 2.5137016e+02,
       2.6765421e+02, 2.8548364e+02, 3.0511923e+02, 3.2686798e+02,
       3.5109348e+02, 3.7822763e+02, 4.0878464e+02, 4.4337769e+02,
       4.8273672e+02, 5.2772803e+02, 5.7937286e+02, 6.3886261e+02,
       7.0756329e+02, 7.8700250e+02, 8.7882526e+02, 9.8470587e+02,
       1.1062042e+03, 1.2445669e+03, 1.4004972e+03, 1.5739464e+03,
       1.7640033e+03, 1.9689442e+03, 2.1864565e+03, 2.4139717e+03,
       2.6490012e+03, 2.8893848e+03, 3.1334048e+03, 3.3797935e+03,
       3.6276704e+03, 3.8764519e+03, 4.1257681e+03, 4.3753926e+03,
       4.6251904e+03, 4.8750835e+03, 5.1250283e+03, 5.3750000e+03],
      dtype=float32)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05

### interp obs vertical grid to model vertical grid

In [6]:
np.shape(temp)

(768, 24, 51, 151)

In [7]:
temp_new = np.ma.zeros([768,35,51,151])
temp_new.mask = True
for mm in range(768):
    for ii in range(51):
        for jj in range(151):
            temp_new[mm,:,ii,jj] = np.interp(z_t[:35],z_r[:],temp[mm,:,ii,jj])

In [8]:
temp_new.mask = temp_new.mask + np.isnan(temp_new)

In [9]:
outfile = f'{path}/EN4_T_verti_regrid_195401-201712_monthly.nc'
root = netCDF4.Dataset(outfile,'w',format='NETCDF4')
root.createDimension('nmon',768)
root.createDimension('nver',35)
root.createDimension('nlat',51)
root.createDimension('nlon',151)
wo2 = root.createVariable('T','f4',('nmon','nver','nlat','nlon'))
wlat = root.createVariable('lat','f4',('nlat',))
wlon = root.createVariable('lon','f4',('nlon',))
wzt = root.createVariable('z_t','f4',('nver',))
wtime = root.createVariable('time','i4',('nmon',))
wo2[:,:,:,:] = temp_new[:,:,:,:]
wlat[:] = lat[:]
wlon[:] = lon[:]
wzt[:] = z_t[:35]
wtime[:] = range(768)[:]
root.close()

### Now GOBAI O2 is in 154 months x 35 vertical layers x nlat x nlon